# NSNSD Night Sky Report Generator 

In [1]:
# Work flow consists of 1.) arcpy steps herein to generate report spatial layers and statistics; 2.) manual map generation
# from spatial layers created in step 1 using the NightSkyReportProcess.aprx map and templates; and 3.) NPS formatted 
# report automation using RMarkdown from NPS night sky metrics data, spatial statistics, spatial layers and mapping generated 
# in step 1 and 2

# IMPORTANT: Place NightSkyReportGenerator_V1 in your park specific project folder (may need to extract from zip)

In [2]:
import sys,os
import arcpy
import glob, os, shutil
from arcpy.sa import *

In [3]:
### START USER INPUT###
# Set report park variables
# Set Unit code
#change four letter acronym unit code for park 
unitcode = "UNIT_CODE = 'PORE'"
park = "PORE"
#Set buffer distance for parks that have monitoring sites on or near the boundary
buffnum = "10"
#Distance search from monitoring sites to cities (census data)
citydist = "300"
units =  " Kilometers"

# Set location where the night sky report generator folder is:
# This is the only path to be edited
repgen = r"C:\Users\Emeyer\Documents\NightSkyMetricsReports\NightSkyReportGenerator"
print(repgen)



### END USER INPUT###

C:\Users\Emeyer\Documents\NightSkyMetricsReports\NightSkyReportGenerator


In [4]:
# Set paths and store names

# Set workspace to project gdb
# This gdb is located in the same folder of the template maps and layers
workspace = repgen + r"\ArcPro\NightSkyReportProcess\NightSkyReportProcess.gdb"

# Location of layer templates
layerworkspace = repgen + r"\ArcPro\NightSkyReportProcess\\"

# This path is where the R.proj and R Markdown file are stored
# report figure locations
figures = repgen + r"\NightSkyReporting\Figures\\"
# this is where excel output will go
parks = repgen + r"\NightSkyReporting\Parks\\"
# This is where the study sites xy data is
studyxl = repgen +"\\NightSkyReporting\\Reference\\NPMapsQuery.xlsx\\Sheet1$"


# Create folders for reporting park to place analysis and maps
arcpy.CreateFolder_management(figures, park)
arcpy.CreateFolder_management(parks, park)

# Set ouput directory to "\NightSkyReportGenerator_V1\NightSkyReporting" on local drive or reporting park directory
output = repgen + r"\NightSkyReporting\Parks\\" + park

#File path for reporting park feature class stored created within this process and stored to project .gdb
ReportPark = workspace + "\ReportPark_"+ park

#File path for sALR raster stored in the project .gdb
sALR = arcpy.Raster(workspace +"\\sALR")

#File path to store sALR zonal stastics table summarized within the reporting park
sALRoutput = workspace + "\\sALRoutput_"+ park

#File path to export sALR zonal stastics table summarized within the reporting park
sALRoutput_xls = output + "\\sALRoutput_" + park +".xls"

#Stored for naming monitoring sites feature class selected within the reporting park
StudySites = workspace + "\\StudySites"

#Stored for naming a buffered boundary around the park to capture study sites on or near the park boundary
buffer = buffnum + units
ReportPark_buffer = workspace + "\\ReportPark_"+ park + "_" + buffnum + "_km"

#Stored for naming a feature class of monitoring sites within the reporting park buffer
ReportPark_StudySites = workspace + "\\ReportPark_StudySites_" + park

# Stored for naming output of distance and angle (Walkers Value calc) from monitoring sites to cities/census data
ReportPark_GenerateNearTable = workspace + "\\ReportPark_GenerateNearTable_"+ park

# Census data stored in .gdb
PopCities2020 = "PopCities2020"
searchradius = citydist + units

# Stored file path for exporting distance, angle, sity, and study site table
ReportPark_NearTable_xlsx = output + "\\ReportPark_NearTable_" + park + ".xlsx"

# Set layer that output symbology will be based on
symbologyLayerPark = layerworkspace + "ReportPark.lyrx"
symbologyLayerSites = layerworkspace + "ReportPark_StudySites.lyrx"

In [5]:
###Create Study Sites feature class that originattes from the NPMapsQuery.xlsx file
###The excel file is updated from the geojson file via R by the project manager

arcpy.env.overwriteOutput = True
arcpy.management.XYTableToPoint(studyxl,
                                StudySites,
                                x_field="longitude", y_field="latitude")



print(arcpy.management.GetCount(StudySites))


633


In [6]:
## Copy common to all figures to new park figure dir

in_data = repgen + r"\NightSkyReporting\Figures\Common"
out_data = figures + park
# Execute Copy of all png files
files = glob.iglob(os.path.join(in_data, "*.PNG"))
for file in files:
    if os.path.isfile(file):
        shutil.copy2(file, out_data)

In [7]:
# Check out any necessary licenses.
arcpy.CheckOutExtension("3D")
arcpy.CheckOutExtension("spatial")
arcpy.CheckOutExtension("ImageExt")
arcpy.CheckOutExtension("ImageAnalyst")

arcpy.ImportToolbox(r"c:\program files\arcgis\pro\Resources\ArcToolbox\toolboxes\Conversion Tools.tbx")

# To allow overwriting outputs change overwriteOutput option to True.
arcpy.env.overwriteOutput = True

In [8]:
# Step 1: Select the park of interest and export to a feature class. Selects park stored from previous steps from the
# Administrative Boundaries (all NPS units) feature class in the project .gdb

# Execute FeatureClassToFeatureClass
arcpy.FeatureClassToFeatureClass_conversion("Administrative_Boundaries_of_National_Park_System_Units", workspace, 
                                            "ReportPark_"+park, unitcode)    



<Result 'C:\\Users\\Emeyer\\Documents\\NightSkyMetricsReports\\NightSkyReportGenerator\\ArcPro\\NightSkyReportProcess\\NightSkyReportProcess.gdb\\ReportPark_PORE'>

In [9]:
# Step 2: Extract sALR model using the reporting park feature class. Complete zonal statistics summarizing ALR model within
# the reporting park. Save table to project .gdb and export to excel in the output directory.

# Process: Zonal Statistics as Table (Zonal Statistics as Table)

arcpy.sa.ZonalStatisticsAsTable(ReportPark, "UNIT_CODE", sALR, sALRoutput, 
                                "DATA", "ALL", "CURRENT_SLICE", [90, 10], "AUTO_DETECT", "ARITHMETIC", 360, "")
   

# Process: Table To Excel (Table To Excel)

arcpy.conversion.TableToExcel(Input_Table=[sALRoutput], Output_Excel_File=sALRoutput_xls)




<Result 'C:\\Users\\Emeyer\\Documents\\NightSkyMetricsReports\\NightSkyReportGenerator\\NightSkyReporting\\Parks\\\\PORE\\sALRoutput_PORE.xls'>

In [10]:
# Step 3: Create a buffer around the reporting park and save to project .gdb

# Process: Buffer (Buffer) (analysis)
arcpy.analysis.Buffer(in_features=ReportPark, out_feature_class=ReportPark_buffer, 
                      buffer_distance_or_field=buffer)


<Result 'C:\\Users\\Emeyer\\Documents\\NightSkyMetricsReports\\NightSkyReportGenerator\\ArcPro\\NightSkyReportProcess\\NightSkyReportProcess.gdb\\ReportPark_PORE_10_km'>

In [11]:
# Step 4: Select study sites within the buffered reporting park feature class

# Process: Select Layer By Location (Select Layer By Location) (management)
tempselect = arcpy.management.SelectLayerByLocation(in_layer=StudySites, overlap_type="WITHIN", select_features=ReportPark_buffer)

# Process: Export Features (Export Features) (conversion)

arcpy.conversion.ExportFeatures(in_features=tempselect, out_features=ReportPark_StudySites)



<Result 'C:\\Users\\Emeyer\\Documents\\NightSkyMetricsReports\\NightSkyReportGenerator\\ArcPro\\NightSkyReportProcess\\NightSkyReportProcess.gdb\\ReportPark_StudySites_PORE'>

In [12]:
# Step 5: Generate distance and angle from monitoring sites to cities within a defined distance. Export output as table to project .gdb


# Process: Select Layer By Attribute (Select Layer By Attribute) (management)
arcpy.management.SelectLayerByAttribute(in_layer_or_view=ReportPark_StudySites, 
                                                                             selection_type="CLEAR_SELECTION")

# Process: Generate Near Table (Generate Near Table) (analysis)

arcpy.analysis.GenerateNearTable(in_features=ReportPark_StudySites, near_features=[PopCities2020], 
                                 out_table=ReportPark_GenerateNearTable, 
                                 search_radius=searchradius, location="LOCATION", 
                                 angle="ANGLE", closest="ALL", closest_count=10000, method="GEODESIC", distance_unit="Kilometers")



<Result 'C:\\Users\\Emeyer\\Documents\\NightSkyMetricsReports\\NightSkyReportGenerator\\ArcPro\\NightSkyReportProcess\\NightSkyReportProcess.gdb\\ReportPark_GenerateNearTable_PORE'>

In [13]:
# Step 6: Join near table created in step 5 to census data to add city name to the table in addition to city id. Previous step
# drops the city name so this step adds the city name back to the table, overwriting the table in Step 5 with the new data
# Process: Join Field (Join Field) (management)
arcpy.management.JoinField(in_data=ReportPark_GenerateNearTable, 
                                                          in_field="NEAR_FID", join_table=PopCities2020, 
                                                          join_field="OBJECTID", 
                                                          fields=["PLACENS", "GEOID", "CLASSFP", "FUNCSTAT", "ALAND", "AWATER", "INTPTLAT", "INTPTLON", "NAME", "P0010001", "H0010001", "H0030002", "P001_calc_", "H0010001", "H0030002", "P001_calc_", "H0010001", "H0030002", "P001_calc_", "P0020002", "P0020002", "P0020003", "P0020004", "P002_calc_", "P002_calc1"])[0]



'C:\\Users\\Emeyer\\Documents\\NightSkyMetricsReports\\NightSkyReportGenerator\\ArcPro\\NightSkyReportProcess\\NightSkyReportProcess.gdb\\ReportPark_GenerateNearTable_PORE'

In [14]:
# Step 7: Joing table from Step 6 to the report park study site feature class and export in excel for analysis in R. 

# Process: Join Field (Join Field) (management)
temptable=arcpy.management.JoinField(in_data=ReportPark_GenerateNearTable,
                           in_field="IN_FID", join_table=ReportPark_StudySites, 
                           join_field="OBJECTID", fields=["SITE_NAME", "DNIGHT", "DSET", "NPS_UNIT", "MID_DATE_LMT"])[0]

# Process: Table To Excel (Table To Excel) (conversion)

arcpy.conversion.TableToExcel(Input_Table=temptable, 
                              Output_Excel_File=ReportPark_NearTable_xlsx, Use_field_alias_as_column_header="NAME", 
                              Use_domain_and_subtype_description="CODE")


<Result 'C:\\Users\\Emeyer\\Documents\\NightSkyMetricsReports\\NightSkyReportGenerator\\NightSkyReporting\\Parks\\\\PORE\\ReportPark_NearTable_PORE.xlsx'>

In [15]:
# Step 8: Repair geometry of park polygon and site points

arcpy.management.RepairGeometry(ReportPark)
arcpy.management.RepairGeometry(ReportPark_StudySites)




<Result 'C:\\Users\\Emeyer\\Documents\\NightSkyMetricsReports\\NightSkyReportGenerator\\ArcPro\\NightSkyReportProcess\\NightSkyReportProcess.gdb\\ReportPark_StudySites_PORE'>

In [16]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
mp = aprx.listMaps("ProcessingMap")[0]

# Remove all layers

for rmlyr in mp.listLayers():
    if rmlyr.name != "":
        mp.removeLayer(rmlyr)
    
# Remove all tables
for rmtbl in mp.listTables():
    if rmtbl.name != "":
        mp.removeTable(rmtbl)

# Save the project
aprx.save()